In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import sys
sys.path.append('../')

from utils.processing import *
from utils.visualisation import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.compose import TransformedTargetRegressor, make_column_transformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import DBSCAN


np.random.seed(0)

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 80)

In [4]:
#https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/
def get_clusters(coords):
    epsilon = 0.3 / 6371.0088
    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    print(num_clusters)

    clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
    
    return db.labels_

def cluster_classifier(coords, labels):
    model = KNeighborsClassifier(algorithm='ball_tree', metric='haversine')
    model.fit(coords, labels)
    return model

In [5]:
# load data set
df = pd.read_csv('../data/train.csv')
df_train_orig, df_validate_orig = train_test_split(df, test_size=0.30)

df_train_orig.iloc[0]

listing_id                                                          224052
title                                         1 bed condo for sale in myra
address                                            9 meyappa chettiar road
property_name                                                         myra
property_type                                                        condo
tenure                                                            freehold
built_year                                                          2024.0
num_beds                                                               1.0
num_baths                                                              1.0
size_sqft                                                              474
floor_level                                                            NaN
furnishing                                                     unspecified
available_unit_types                                         1, 2, 3, 4 br
total_num_units          

In [6]:
def prepare_data_for_regression(df, drop_columns=[], classifier=None):
    if classifier != None:
        df['labels'] = classifier.predict(df[['lat','lng']])
        
    adfs = read_aux_csv('../data')
    df = join_aux(df, adfs)
    df = df.drop(columns=drop_columns)

    return df

def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    score = np.sqrt(mse)
    return score

def rmse_scorer():
    return make_scorer(rmse, greater_is_better=False)

# Constants
initial_drop_columns = [
    'address',
    'title',
    'listing_id',
    'property_name',
    'available_unit_types',
    'property_details_url',
    'elevation',
    'tenure',
    'property_type',
    'floor_level',
    'furnishing',
]

drop_columns = initial_drop_columns + [
    'lat',
    'lng',
    'floor_level_ground',
    'floor_level_high',
    'floor_level_low',
    'floor_level_mid',
    'floor_level_penthouse',
    'floor_level_top',
    'furnishing_partial',
    'furnishing_unfurnished',
    'furnishing_unspecified',
    'line_cc',
    'line_ce',
    'line_cg',
    'line_dt',
    'line_ew',
    'line_ne',
    'line_ns',
    'line_te',
    'gep_pri_sch_within_1km',
    'gep_pri_sch_within_1km_2km',
    'cc_type_CR',
    'cc_type_IEBP',
    'cc_type_IHL',
]

In [7]:
labels = get_clusters(df_train_orig[['lat', 'lng']])
classifier = cluster_classifier(df_train_orig[['lat','lng']], labels)

289


In [8]:
df_train = df_train_orig.copy()
df_train['labels'] = labels
df_train = preprocess(df_train)
df_train = prepare_data_for_regression(df_train, drop_columns=drop_columns)
df_train

,built_year,num_beds,num_baths,size_sqft,total_num_units,subzone,planning_area,price,labels,property_type_private,property_type_public,tenure_high_year,tenure_low_year,nearest_mrt_distance_in_km,nearest_pri_sch_distance_in_km,nearest_gep_pri_sch_distance_in_km,gep_pri_sch_outside_2km,pri_sch_within_500m,pri_sch_outside_500m,nearest_com_centre_distance_in_km,cc_type_BN,nearest_mall_distance_in_km,area_size,population,density,region
0,2024.0,1.0,1.0,474.0,85.0,potong pasir,toa payoh,1060900.0,0,1,0,1,0,0.131235,0.342137,3.934774,1,1,0,2.251585,1,0.100566,0.6391,11720,18338.288218,c
1,2012.0,2.0,2.0,915.0,74.0,lorong ah soo,hougang,1029900.0,1,1,0,0,1,0.780849,0.366408,2.363387,1,1,0,0.694122,0,0.339265,1.5155,32320,21326.294952,ne
2,2025.0,4.0,4.0,4717.0,6.0,mountbatten,marine parade,8059800.0,2,1,0,1,0,0.973647,0.998307,2.130345,1,0,1,1.952462,1,1.000584,1.7119,9980,5829.779777,c
3,1976.0,3.0,1.0,699.0,NaN,farrer court,bukit timah,554400.0,3,0,1,0,1,0.212855,0.546077,0.546077,0,0,1,2.066441,1,1.325490,0.5588,6180,11059.413028,c
4,2022.0,3.0,3.0,1249.0,52.0,nassim,tanglin,3738000.0,4,1,0,1,0,0.760161,0.466849,0.466849,0,1,0,1.340886,1,0.977231,2.0961,9520,4541.768045,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14093,1992.0,3.0,2.0,979.0,70.0,yishun south,yishun,514500.0,13,0,1,0,0,0.675683,0.287658,6.676536,1,1,0,3.438561,0,0.454060,1.3402,42240,31517.683928,n
14094,2024.0,2.0,2.0,689.0,200.0,mountbatten,marine parade,2052600.0,26,1,0,1,0,1.368677,1.442544,2.465268,1,0,1,2.436429,1,1.402825,1.7119,9980,5829.779777,c
14095,2018.0,3.0,3.0,1518.0,1042.0,central subzone,downtown core,4122000.0,62,1,0,0,1,0.151672,1.486155,2.350847,1,0,1,1.191965,0,0.530202,0.9793,880,898.601042,c
14096,2022.0,3.0,3.0,1249.0,117.0,nassim,tanglin,3934400.0,4,1,0,1,0,0.810220,0.635320,0.635320,0,0,1,1.492339,1,0.883319,2.0961,9520,4541.768045,c


In [9]:
X_train = df_train.drop('price', axis=1)
y_train = df_train['price']

In [10]:
ct = make_column_transformer(
                    (make_pipeline(KNNImputer(), StandardScaler()), ['num_beds', 'num_baths', 'built_year', 'total_num_units']),
                    (StandardScaler(), ['nearest_mrt_distance_in_km', 'nearest_pri_sch_distance_in_km', 'nearest_gep_pri_sch_distance_in_km', 'nearest_com_centre_distance_in_km', 'nearest_mall_distance_in_km', 'area_size', 'population', 'density']),
                    (OneHotEncoder(sparse=False, handle_unknown='ignore'), ["region", "subzone", "planning_area"]),
                    remainder='passthrough')

def ttr(pipeline):
    return TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())

def transform_data(df):
    p = make_pipeline(ct)
    p.fit(df)
    
    return pd.DataFrame(data=p.transform(df), columns=p.get_feature_names_out())

In [11]:
t_X_train = transform_data(X_train)
t_X_train.head()

,pipeline__num_beds,pipeline__num_baths,pipeline__built_year,pipeline__total_num_units,standardscaler__nearest_mrt_distance_in_km,standardscaler__nearest_pri_sch_distance_in_km,standardscaler__nearest_gep_pri_sch_distance_in_km,standardscaler__nearest_com_centre_distance_in_km,standardscaler__nearest_mall_distance_in_km,standardscaler__area_size,standardscaler__population,standardscaler__density,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__subzone_admiralty,onehotencoder__subzone_alexandra hill,onehotencoder__subzone_alexandra north,onehotencoder__subzone_aljunied,onehotencoder__subzone_anak bukit,onehotencoder__subzone_anchorvale,onehotencoder__subzone_ang mo kio town centre,onehotencoder__subzone_anson,...,onehotencoder__planning_area_queenstown,onehotencoder__planning_area_river valley,onehotencoder__planning_area_rochor,onehotencoder__planning_area_seletar,onehotencoder__planning_area_sembawang,onehotencoder__planning_area_sengkang,onehotencoder__planning_area_serangoon,onehotencoder__planning_area_singapore river,onehotencoder__planning_area_southern islands,onehotencoder__planning_area_tampines,onehotencoder__planning_area_tanglin,onehotencoder__planning_area_tengah,onehotencoder__planning_area_toa payoh,onehotencoder__planning_area_woodlands,onehotencoder__planning_area_yishun,remainder__size_sqft,remainder__labels,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__gep_pri_sch_outside_2km,remainder__pri_sch_within_500m,remainder__pri_sch_outside_500m,remainder__cc_type_BN
0,-1.659098,-1.126635,0.860023,-0.878097,-1.186920,-0.747518,0.174252,0.311802,-1.301452,-0.428034,-0.442971,0.292401,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,474.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1,-0.876694,-0.442865,0.092996,-0.911730,-0.017542,-0.702367,-0.388648,-1.145113,-0.852498,-0.092842,0.434958,0.542237,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,915.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
2,0.688114,0.924676,0.923942,-1.119644,0.329516,0.473157,-0.472128,0.031990,0.391341,-0.017727,-0.517127,-0.753474,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4717.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,-0.094290,-1.126635,-2.208085,-0.531981,-1.039994,-0.368129,-1.039642,0.138611,1.002440,-0.458745,-0.679075,-0.316208,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,699.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
4,-0.094290,0.240906,0.732185,-0.978996,-0.054783,-0.515517,-1.068023,-0.540103,0.347419,0.129216,-0.536731,-0.861168,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1249.0,4.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [12]:
t_X_train[t_X_train.isna().any(axis=1)]

,pipeline__num_beds,pipeline__num_baths,pipeline__built_year,pipeline__total_num_units,standardscaler__nearest_mrt_distance_in_km,standardscaler__nearest_pri_sch_distance_in_km,standardscaler__nearest_gep_pri_sch_distance_in_km,standardscaler__nearest_com_centre_distance_in_km,standardscaler__nearest_mall_distance_in_km,standardscaler__area_size,standardscaler__population,standardscaler__density,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__subzone_admiralty,onehotencoder__subzone_alexandra hill,onehotencoder__subzone_alexandra north,onehotencoder__subzone_aljunied,onehotencoder__subzone_anak bukit,onehotencoder__subzone_anchorvale,onehotencoder__subzone_ang mo kio town centre,onehotencoder__subzone_anson,...,onehotencoder__planning_area_queenstown,onehotencoder__planning_area_river valley,onehotencoder__planning_area_rochor,onehotencoder__planning_area_seletar,onehotencoder__planning_area_sembawang,onehotencoder__planning_area_sengkang,onehotencoder__planning_area_serangoon,onehotencoder__planning_area_singapore river,onehotencoder__planning_area_southern islands,onehotencoder__planning_area_tampines,onehotencoder__planning_area_tanglin,onehotencoder__planning_area_tengah,onehotencoder__planning_area_toa payoh,onehotencoder__planning_area_woodlands,onehotencoder__planning_area_yishun,remainder__size_sqft,remainder__labels,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__gep_pri_sch_outside_2km,remainder__pri_sch_within_500m,remainder__pri_sch_outside_500m,remainder__cc_type_BN


In [13]:
def run_models(models, X, y):
    model_scores = dict()
    
    for i, model in enumerate(models):
        regr = ttr(make_pipeline(ct, model))
  
        scores = cross_validate(regr, X, y, cv=5, scoring=rmse_scorer())
        regr.fit(X, y)
        model_scores[model.__class__.__name__ + str(i)] = [regr, scores]
        print(model.__class__.__name__, scores['test_score'].mean())
        
    return model_scores

In [14]:
models = [
            xgb.XGBRegressor(),
            #xgb.XGBRegressor(n_estimators=150, max_depth=6, learning_rate=0.3),
            #xgb.XGBRegressor(n_estimators=150, max_depth=5, learning_rate=0.3),
        ]


model_scores = run_models(models, X_train, y_train)

XGBRegressor -1877422.1788640108


In [15]:
y_validate = df_validate_orig['price']
X_validate = prepare_data_for_regression(preprocess(df_validate_orig.drop('price', axis=1), is_target=True), drop_columns=drop_columns, classifier=classifier)

t_X_v = transform_data(X_validate)
t_X_v.head()

,pipeline__num_beds,pipeline__num_baths,pipeline__built_year,pipeline__total_num_units,standardscaler__nearest_mrt_distance_in_km,standardscaler__nearest_pri_sch_distance_in_km,standardscaler__nearest_gep_pri_sch_distance_in_km,standardscaler__nearest_com_centre_distance_in_km,standardscaler__nearest_mall_distance_in_km,standardscaler__area_size,standardscaler__population,standardscaler__density,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__subzone_admiralty,onehotencoder__subzone_alexandra hill,onehotencoder__subzone_alexandra north,onehotencoder__subzone_aljunied,onehotencoder__subzone_anak bukit,onehotencoder__subzone_anchorvale,onehotencoder__subzone_ang mo kio town centre,onehotencoder__subzone_anson,...,onehotencoder__planning_area_queenstown,onehotencoder__planning_area_river valley,onehotencoder__planning_area_rochor,onehotencoder__planning_area_seletar,onehotencoder__planning_area_sembawang,onehotencoder__planning_area_sengkang,onehotencoder__planning_area_serangoon,onehotencoder__planning_area_singapore river,onehotencoder__planning_area_southern islands,onehotencoder__planning_area_tampines,onehotencoder__planning_area_tanglin,onehotencoder__planning_area_tengah,onehotencoder__planning_area_toa payoh,onehotencoder__planning_area_woodlands,onehotencoder__planning_area_yishun,remainder__size_sqft,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__labels,remainder__gep_pri_sch_outside_2km,remainder__pri_sch_within_500m,remainder__pri_sch_outside_500m,remainder__cc_type_BN
0,0.682822,0.916767,0.790116,-1.000407,-0.485358,0.438822,-0.553770,-0.650078,-0.555592,0.058527,0.106832,-0.174706,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2153.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
1,-0.862389,-1.106424,0.917894,-0.143403,-0.081178,0.909779,-0.751929,-0.400462,0.083827,0.091878,-0.667007,-0.972863,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,614.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,1.0
2,-0.089783,-0.432027,0.406781,0.160307,-0.088027,-0.923569,2.873543,-0.428278,-0.240812,0.282934,3.231257,1.995580,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,991.0,0.0,1.0,0.0,1.0,70.0,1.0,1.0,0.0,0.0
3,2.228032,0.916767,-2.148782,-0.997400,0.091244,-0.679120,-0.776326,-0.975115,-1.060358,0.050365,0.427826,0.167178,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3240.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,-0.089783,-0.432027,0.087336,1.187509,0.828457,-0.810085,1.150104,0.963756,-0.032994,-0.007991,1.617441,1.622717,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,990.0,0.0,1.0,0.0,1.0,13.0,1.0,1.0,0.0,0.0


In [16]:
t_X_v[t_X_v.isna().any(axis=1)]

,pipeline__num_beds,pipeline__num_baths,pipeline__built_year,pipeline__total_num_units,standardscaler__nearest_mrt_distance_in_km,standardscaler__nearest_pri_sch_distance_in_km,standardscaler__nearest_gep_pri_sch_distance_in_km,standardscaler__nearest_com_centre_distance_in_km,standardscaler__nearest_mall_distance_in_km,standardscaler__area_size,standardscaler__population,standardscaler__density,onehotencoder__region_c,onehotencoder__region_e,onehotencoder__region_n,onehotencoder__region_ne,onehotencoder__region_w,onehotencoder__subzone_admiralty,onehotencoder__subzone_alexandra hill,onehotencoder__subzone_alexandra north,onehotencoder__subzone_aljunied,onehotencoder__subzone_anak bukit,onehotencoder__subzone_anchorvale,onehotencoder__subzone_ang mo kio town centre,onehotencoder__subzone_anson,...,onehotencoder__planning_area_queenstown,onehotencoder__planning_area_river valley,onehotencoder__planning_area_rochor,onehotencoder__planning_area_seletar,onehotencoder__planning_area_sembawang,onehotencoder__planning_area_sengkang,onehotencoder__planning_area_serangoon,onehotencoder__planning_area_singapore river,onehotencoder__planning_area_southern islands,onehotencoder__planning_area_tampines,onehotencoder__planning_area_tanglin,onehotencoder__planning_area_tengah,onehotencoder__planning_area_toa payoh,onehotencoder__planning_area_woodlands,onehotencoder__planning_area_yishun,remainder__size_sqft,remainder__property_type_private,remainder__property_type_public,remainder__tenure_high_year,remainder__tenure_low_year,remainder__labels,remainder__gep_pri_sch_outside_2km,remainder__pri_sch_within_500m,remainder__pri_sch_outside_500m,remainder__cc_type_BN


In [17]:
for k, v in model_scores.items():
    regr, scores = v
    predict_validate = regr.predict(X_validate)

    print(k, rmse(y_validate, predict_validate))

XGBRegressor0 1984180.1929618872


In [18]:
# Retrain using best model
fmodel = make_pipeline(ct, LinearDiscriminantAnalysis(), xgb.XGBRegressor())
#fmodel = pipelined_model(xgb.XGBRegressor(n_estimators=150, max_depth=5, learning_rate=0.3))
#fmodel = pipelined_model(xgb.XGBRegressor(n_estimators=50,reg_lambda=10,max_depth=1))

df_ftrain = pd.read_csv('../data/train.csv')

labels = get_clusters(df_ftrain[['lat', 'lng']])
classifier = cluster_classifier(df_ftrain[['lat','lng']], labels)

df_ftrain['labels'] = labels
df_ftrain = preprocess(df_ftrain)
df_ftrain = prepare_data_for_regression(df_ftrain, drop_columns=drop_columns)
df_ftrain.head()

X_ftrain = df_ftrain.drop('price', axis=1)
y_ftrain = df_ftrain['price']

fmodel.fit(X_ftrain, y_ftrain)
predict_ftrain = fmodel.predict(X_ftrain)
print(rmse(y_ftrain, predict_ftrain))

234
424011.87731550564


In [19]:
df_test = preprocess(pd.read_csv('../data/test.csv'), is_target=True)

# further preprocessing
X_test = prepare_data_for_regression(df_test, drop_columns=drop_columns, classifier=classifier)

y_predict = fmodel.predict(X_test)

X_test['Predicted'] = y_predict

In [20]:
y_predict

array([1213262.4, 1442695.6, 1373835.4, ..., 3458200.5,  537871.9,
       4181336.2], dtype=float32)

In [21]:
submission = X_test[['Predicted']]
submission.to_csv('submission3.csv', index=True, index_label='id', header=True, columns=['Predicted'])